# Coeff_finder

#### Finds the offset between the theoretical flux and measured flux of a given object, using SIMBAD and photometric data respectively

Test object must have a spectral type on SIMBAD of format "XY" where X is spectral class, and Y is sub-class, for example, G2.

Theoretical fluxes are calculated for all filters specified in 'filters.dat', located at '.'. Measured fluxes are carried out for all files within './testfiles/'.

In [1]:
from astroquery.simbad import Simbad
import numpy as np
from PyAstronomy import pyasl
import scipy.integrate as integ
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
from random import gauss
import sewpy
from astropy.io import fits
from os import listdir
from astropy.coordinates import SkyCoord
from astropy import units as u

Specify camera and target name, and .fits file location:

In [13]:
camera_name = 'PL09000'
target_name = 'LTT1788'
directory = './ltt1788-red/'
files = listdir(directory)

Obtain filters and set site and camera data

In [14]:
fcentres, fwidths = np.genfromtxt('filters.dat',unpack=True, usecols=(1,2)) #load filter data from file
fnames = np.genfromtxt('filters.dat',unpack=True, usecols=(0), dtype='str').tolist()
pixel_width_m = 12 #pixel width in um
plate_scale = 0.621 #plate scale in as/um
pixel_width_as = pixel_width_m*plate_scale #pixelwidth in as
site_seeing = 3 #seeing in as


wave = np.arange(10e-10, 25000e-10,10e-10) #initialise wavelength range

Define SExtractor values

In [15]:
sew = sewpy.SEW(params=['X_IMAGE','Y_IMAGE','FWHM_WORLD','ALPHA_J2000','DELTA_J2000','FLUX_BEST'],
                config={'DETECT_MINAREA':20, 'PHOT_APERTURES':'5,10,20,50'},
                sexpath='sex')

Create Simbad object, and obtain coordinate and B magnitude

In [16]:
custom_sim = Simbad()
#Simbad.list_votable_fields()
custom_sim.add_votable_fields('flux(B)','sp','coordinates') # add desired fields
simdata = custom_sim.query_object(target_name) #query SIMBAD
#extract coords
raw_ra = simdata['RA'][0]
raw_dec = simdata['DEC'][0]

c = SkyCoord(raw_ra,raw_dec, unit=(u.hourangle,u.deg)) #create SkyCoord object
#convert coords to degrees
ra = c.ra.deg
dec = c.dec.deg
Bmag = simdata[0]['FLUX_B']

Load spectral type/temp data table from file and specify known standard filter data

In [17]:
types = np.genfromtxt('colour-temp.txt',unpack=True,usecols=(0),dtype='str') #load spectral types
temps = np.genfromtxt('colour-temp.txt',unpack=True,usecols=(1)) #load corresponding temperatures

B_zp = -20.45 #known zero point for B

#standard filters, centres, widths, and zero points
filters = ['U','B','V','R','I','J','H','K']
centres = [3650,4450,5510,6580,8060,12200,16300,21900]
widths = [660,940,880,1380,1490,2130,3070,3900]
zeros = [-20.94,-20.45,-21.12,-21.61,-22.27,-23.80,-24.80,-26.00]

Extract target spectral type, convert to temperatre using data table and define Planck function.

Define polynomial that describes proportionality constant between calculated and actual zero points for a range of wavelengths:

In [18]:
sp = str(simdata[0]['SP_TYPE'])[2:3]#obtain 2-character spectral type from data table
temp_val = temps[np.where(types == sp+'0V')[0][0]]  #convert spectral type to tempterature using lookup table
#lookup table only has V spectral classes, so add V as an approximation
planck = pyasl.planck(temp_val,lam=wave) #define planck function from temperature

alphas=[] #initialise array of alphas
order = 3 #order of polynomial to use, determined by analysis in alpha.ipynb
for i in range(0,len(filters)): #loop across filters
    loop_band = np.zeros(len(wave)) #new zeros
    #determine edges of each waveband
    loop_band_min = (centres[i] - widths[i]/2)*1e-10 
    loop_band_max = (centres[i] + widths[i]/2)*1e-10
    #find corresponding indexes
    loop_start_index = (np.abs(wave - loop_band_min)).argmin()
    loop_end_index = (np.abs(wave - loop_band_max)).argmin()

    #set values within indexes to one
    for j in range(loop_start_index, loop_end_index):
        loop_band[j] = 1.0

    loop_trans = planck*loop_band #find transmission in each standard filter
    loop_flux = integ.simps(loop_trans, wave, dx=10e-10) #find total flux in each standard filter

    z_calc = 2.5*math.log10(loop_flux) #find zero point
    new_alpha = zeros[i]/z_calc #find proportionalty between calculated zero and known zero
    alphas.append(new_alpha) #add to array
poly = np.poly1d(np.polyfit(centres,alphas,order)) #fit polynomial to the alpha values


Find flux in B band from defined Planck function:

In [19]:
Bmid = 4450 #centre
Bwidth = 940 #width

Bband = np.zeros(len(wave)) #new zeros
#find edges of band
Bband_min = (Bmid - Bwidth/2)*1e-10
Bband_max = (Bmid + Bwidth/2)*1e-10
#find corresponding indexes
Bstart_index = (np.abs(wave - Bband_min)).argmin()
Bend_index = (np.abs(wave - Bband_max)).argmin()

#set values within indexes to one
for i in range(Bstart_index, Bend_index):
    Bband[i] = 1.0
    
Btrans = planck*Bband #find band transmission
Bflux = integ.simps(Btrans, wave, dx=10e-10) #integrate to get flux in band

Load CCD efficiency polynomial from file and obtain profile across wavelength range

In [20]:
ccd_poly_coef = np.genfromtxt('./CCD-effs/'+camera_name+'.poly',usecols=(1))
ccd_poly = np.poly1d(ccd_poly_coef)
ccd_res_curve = ccd_poly(wave)

Calculate theoretical fluxes in each desired filter band through use of the flux in the B band and filter zero points:

In [21]:
#theory_fluxes = [] #initialise array

#for i in range(0,len(fcentres)):
mid = fcentres
wid = fwidths
band = np.zeros(len(wave)) #create zeros
#determine edges of waveband
band_min = (mid - wid/2)*1e-10
band_max = (mid + wid/2)*1e-10
#find corresponding indexes in array
start_index = (np.abs(wave - band_min)).argmin()
end_index = (np.abs(wave - band_max)).argmin()

#set values within indexes to one
for j in range(start_index, end_index):
    band[j] = 1.0

alpha = poly(mid) #find alpha for given filter centre
#print('alpha: '+str(alpha))

trans = planck*band #find transmission in given filter
flux = integ.simps(trans, wave, dx=10e-10) #find total flux
z_calc = 2.5*math.log10(flux) #calculate zero point
zp = alpha*z_calc #use correction to obtain actual zero point


BfoFf = Bflux/flux #calculate ratio of flux in B and flux in filter
Dzp = B_zp - zp #delta zero points
Fmag = Bmag + 2.5*math.log10(BfoFf) - Dzp #calcualte magnitude in filter

scaled_Fflux = 10**((zp - Fmag)/2.5) #convert to flux
    
telescope_eff = 0.96*0.96*(1-0.005)
flux = scaled_Fflux*telescope_eff

telescope_r = 0.305
telescope_area = np.pi*telescope_r*telescope_r
telescope_obs = 0.47

flux = flux*telescope_area*telescope_obs

ccd_res_band = ccd_res_curve[start_index:end_index] #create CCD response profile in filter band
ccd_res = np.mean(ccd_res_band) #take average value
ccd_flux = flux*ccd_res #calculate flux after ccd response

total_flux = ccd_flux
gauss_amp = total_flux/(2*np.pi*site_seeing*site_seeing) #calculate amplitude of a 2d Gaussian that encloses the total flux
def integrand(x, y, A, sigma):
    """Defines the function of a 2d Gaussian function that encloses the total flux
    :param x: x coordinates to be integrated over
    :param y: y coordinates to be intagrated over
    :param A: amplitude of the Gaussian curve
    :param sigma: width of the Gaussian, specified by the site seeing
    :return: function of 2d Gaussian
    """
    return A*np.exp(-1/(2*sigma*sigma)*(x*x+y*y))
#carry out integral to find total flux on a single pixel
I = integ.dblquad(integrand,-pixel_width_as/2,pixel_width_as/2,-pixel_width_as/2,pixel_width_as/2, args=(gauss_amp,site_seeing))[0]
print(I)
#theory_fluxes.append(I) #add to array
theory_flux = I

1.0914277104706379e-15


Measure the flux of the target star on a test frame in counts and convert to flux, accounting for CCD efficiency and gain, exposure time of the frame and energy of individual photons:

In [24]:
measured_fluxes =[] #initialise array

#loop over all files
for file in files:
    #run SExtractor and extract table
    sewdata = sew(directory+file)
    sewtable = sewdata['table']
    #extract data columns
    flux_column = sewtable['FLUX_BEST']
    ra_column = sewtable['ALPHA_J2000']
    dec_column = sewtable['DELTA_J2000']
    #obtain index of entry with closest coordinates to coordinates form SIMBAD
    suitable = np.where(np.logical_and(np.abs(ra_column - ra) < 0.01, np.abs(dec_column - dec) < 0.01))[0]
    
    #load fits header and obtain exposure time and filter
    fits_header = fits.open(directory+file)[0].header
    exp_time = fits_header['EXPTIME']
    #filt = fits_header['FILTER']
    filt = 'r\''
    
    filter_index = fnames.index(filt) #find filter index in list
    #extract parameters
    mid = fcentres
    wid = fwidths
    fband = np.arange((mid-wid/2)*1e-10, (mid+wid/2)*1e-10, 10e-10) #specify waveband
    ccd_curve = ccd_poly(fband) #create CCD efficiency polynomial in range
    ccd_avgres = np.mean(ccd_curve) #take average
    
    #calculate energy of a single photon
    h = 6.634e-34
    c = 3e8
    avgE = h*c/(mid*1e-10)

    counts = flux_column[suitable][0] #extract flux in counts from SExtractor
    gain = 1.4 #CCD gain e-/counts
    electrons = counts*gain # e-
    photons = electrons/ccd_avgres # photons
    flux_tot = avgE*photons # J
    flux_rate = flux_tot/exp_time # J/s (W)
    #print(flux_rate)
    measured_fluxes.append(flux_rate)

Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T04-02-15_ltt1788_Red_T-25_30s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T04-07-17_ltt1788_Red_T-25_30s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T04-04-46_ltt1788_Red_T-25_30s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T04-03-30_ltt1788_Red_T-25_30s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T04-06-02_ltt1788_Red_T-25_30s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T03-37-17_ltt1788_Red_T-25_60s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T03-40-48_ltt1788_Red_T-25_60s.cat.txt already exists, I will overwrite it
Output catalog /tmp/sewpy_workdir__zqcw3qi/2018-10-17T03-39-02_ltt1788_Red_T-25_60s.cat.txt already exis

In [25]:
for i in range(0,len(measured_fluxes)):
    print(theory_flux,measured_fluxes[i], theory_flux/measured_fluxes[i])

1.0914277104706379e-15 7.9565553171709185e-16 1.371733956421122
1.0914277104706379e-15 7.871208344880912e-16 1.3866075736395091
1.0914277104706379e-15 7.908460180343136e-16 1.380076128072864
1.0914277104706379e-15 7.744455834201943e-16 1.409302001117433
1.0914277104706379e-15 7.857643651942664e-16 1.3890012818293198
1.0914277104706379e-15 7.76779663914441e-16 1.4050673069510917
1.0914277104706379e-15 7.83859038885674e-16 1.3923775274980568
1.0914277104706379e-15 7.891308838891272e-16 1.3830756503809365
1.0914277104706379e-15 7.793921059021667e-16 1.400357666193298
1.0914277104706379e-15 7.94414703954721e-16 1.3738765219693687
